In [24]:
import pandas as pd
import numpy as np
import io
import json
import requests



def transformed_data():


    url = 'https://raw.githubusercontent.com/shughestr/PIMS_2020_Real_Estate_data/master/sample_clean.csv'
    df = pd.read_csv(url, error_bad_lines=False)




### Normalise absolute value of crime variables and income recipients by the respective population 

    for col in ['saf1','saf2','saf3','saf4','saf5','saf6','saf7','saf8','inc2']:
        df[col] = 100 * df[col] / df['pop1'] 




### Adding vacancy rate column from other census dataset

    data2 = requests.get('https://data.calgary.ca/resource/set9-futw.json')
    df2 = pd.DataFrame(json.loads(data2.text))

    df2 = df2.loc[df2['dwelling_type_code'].isin([str(x) for x in range(1,11)])]

    df2 = df2.drop(labels = ['census_year', 'community','ward','dwelling_type','dwelling_type_code','dwelling_type_description'],axis=1)

    df2[['dwelling_cnt', 'resident_cnt', 'ocpd_dwelling_cnt',
       'vacant_dwelling_cnt', 'ocpd_ownership_cnt', 'renovation_dwelling_cnt',
       'under_const_dwelling_cnt', 'inactive_cnt', 'other_purpose_cnt']] = df2[['dwelling_cnt', 'resident_cnt', 'ocpd_dwelling_cnt',
       'vacant_dwelling_cnt', 'ocpd_ownership_cnt', 'renovation_dwelling_cnt',
       'under_const_dwelling_cnt', 'inactive_cnt', 'other_purpose_cnt']].astype(int)

    df2=df2.groupby(['code']).sum()

    df2['vacancy_rate'] = df2['vacant_dwelling_cnt']/(df2['vacant_dwelling_cnt']+df2['ocpd_dwelling_cnt']) * 100

    df2vac = df2['vacancy_rate']

    vacdict = df2vac.to_dict()

    df['vacancy_rate'] = 0

    for x in vacdict:
        df['vacancy_rate'] = np.where(df['COMM_CODE']==x, vacdict[x],df['vacancy_rate'])
    url = 'https://raw.githubusercontent.com/neha0812/BCFFSA-project/master/walk_score.csv'
    df3 = pd.read_csv(url, error_bad_lines=False)
# making the entries of both the csv files in lowercase for mapping purpose.
    df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)  
    df3 = df3.apply(lambda x: x.str.lower() if x.dtype == "object" else x)  
    df3 = df3.set_index('Name')
#Adding walk score, bike score, transit score and population for different community in calgary
    df['walk_score_comm'] = df['COMM_NAME'].map(df3['Walk Score'])
    df['transit_score_comm'] = df['COMM_NAME'].map(df3['Transit Score'])
    df['bike_score_comm'] = df['COMM_NAME'].map(df3['Bike Score'])
    df['comm_polulation'] = df['COMM_NAME'].map(df3['Population'])    
    return df
df= transformed_data()
df

,ADDRESS,ASSESSED_VALUE,ASSESSMENT_CLASS,ASSESSMENT_CLASS_DESCRIPTION,COMM_CODE,COMM_NAME,LATITUDE,LONGITUDE,ROLL_NUMBER,Year,...,own3,own4,lab1,lab2,lab3,vacancy_rate,walk_score_comm,transit_score_comm,bike_score_comm,comm_polulation
0,4 saddlemead rd ne,343500.0,re,residential,sad,saddle ridge,51.126750,-113.944255,4017208,2017,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
1,4 saddlemead rd ne,345500.0,re,residential,sad,saddle ridge,51.126759,-113.944253,4017208,2018,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
2,4 saddlemead rd ne,323500.0,re,residential,sad,saddle ridge,51.126759,-113.944253,4017208,2019,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
3,4 saddlemead rd ne,315500.0,re,residential,sad,saddle ridge,51.126759,-113.944253,4017208,2020,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
4,70 saddleback rd ne,393500.0,re,residential,sad,saddle ridge,51.128776,-113.948802,4034203,2017,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
5,70 saddleback rd ne,397000.0,re,residential,sad,saddle ridge,51.128774,-113.948804,4034203,2018,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
6,70 saddleback rd ne,390000.0,re,residential,sad,saddle ridge,51.128774,-113.948804,4034203,2019,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
7,70 saddleback rd ne,382500.0,re,residential,sad,saddle ridge,51.128774,-113.948804,4034203,2020,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
8,81 saddlehorn cr ne,524000.0,re,residential,sad,saddle ridge,51.130782,-113.951447,4057006,2017,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
9,81 saddlehorn cr ne,530000.0,re,residential,sad,saddle ridge,51.130773,-113.951477,4057006,2018,...,7.8,0.749317,72.1,9.9,0.273286,7.267587,32.0,53.0,48.0,"12,854"
